In [ ]:
import sys, os

MARIPOSA_PATH = "/home/yizhou7/mariposa/"

sys.path.append(MARIPOSA_PATH + "src")
os.chdir(MARIPOSA_PATH)

from debugger.strainer import DebugStatus
from bench.viewer import BenchViewer
from bench.factory import get_mariposa_viewer

from z3 import *

In [ ]:
only_inst_fixable = set()
for q in round1.status[DebugStatus.FIX_FOUND]:
    stabilized = round1[q].report.stabilized
    print(stabilized)
    print()
    # for fix in .iterrows():
    # if "erase" in stabilized.action.values:
    #     continue
    # if "skolemize" in stabilized.action.values:
    #     continue
    # only_inst_fixable.add(q)
# print(len(only_inst_fixable))
# print(only_inst_fixable)
# only_inst_fixable = list(only_inst_fixable)

In [ ]:
q = only_inst_fixable[1]

def find_quant(qname, query):
    with open(query, 'r') as f:
        lines = f.readlines()
    target_1 = ":qid " + qname + " "
    
    for i, line in enumerate(lines):
        if target_1 in line:
            return line
    return None

def check_fix(q):
    dbg = round1[q]
    stabilized = dbg.report.stabilized
    for fix in stabilized.iterrows():
        fix = fix[1]
        quant = find_quant(fix.qname, dbg.tracker.orig_path)
        if quant.startswith("(assert (! (forall "):
            return True
        if quant.startswith("(assert (! (=> true (forall"):
            return True
    return False

double_check = []
for q in only_inst_fixable:
    if not check_fix(q):
        double_check.append(q)

    # print(fix.action)
    # print(fix.qname)
    # print(dbg.tracker.orig_path)
    # print(diff_files(dbg.tracker.orig_path, fix.edit_path))
print(len(double_check))

In [ ]:
for q in double_check:
    dbg = round1[q]
    stabilized = dbg.report.stabilized
    for fix in stabilized.iterrows():
        quant = find_quant(fix.qname, dbg.tracker.orig_path)
        print(dbg.tracker.orig_path)
        print(quant[:100])
        print(fix.qname, fix.action)
        print(fix.edit_path)
        print("")
        # diff = diff_files(dbg.tracker.orig_path, fix.edit_path)
        # print(diff)
        break

In [ ]:
s = Solver()
s.from_file("dbg/36a1859f27/orig.smt2")
gg = s.assertions()[-1].sexpr()
assert "mariposa_qid_961" in gg

g = z3.Goal()
g.add(s.assertions()[-1])
res = z3.Tactic("snf")(g)
assert "mariposa_qid_961" in res.sexpr()
print(res.sexpr())